<a href="https://colab.research.google.com/github/ck1972/Hands-On-GeoAI1/blob/main/GeoAI_Lab_2a_Preparing_Data_for_Land_Cover_Classification_in_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2a. Preparing Data for Land Cover Classification in GEE**


# Initialize and Authenticate Earth Engine
To get started with Google Earth Engine (GEE), you need to initialize and authenticate the Earth Engine API.

In [ ]:
# Import the API
import ee

# Import the geemap library
import geemap

Next, initialize the Earth Engine API. You must initialize the API to use Earth Engine functionalities. This involves authenticating your session and initializing the library. When you run the ee.Initialize() command for the first time, you might be prompted to authenticate your session. This will open a web browser window where you need to log in with your Google account and grant Earth Engine access.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-kamusoko-test') # Change to your EE project

## Define the boundary
First, we define an area of interest by creating a boundary using FAO dataset from GEE catalog.

In [ ]:
# Load GADM Level 2 data
gadm2 = ee.FeatureCollection("FAO/GAUL/2015/level2")

# Filter by name
boundary = gadm2.filter(ee.Filter.eq('ADM2_NAME', 'Johor Bahru'))

# Check boundary info
print(boundary.getInfo())

# Create an interactive map centered on Colombo (approximate center)
Map = geemap.Map(center=[1.4854, 103.7618], zoom=11)

# Add the Colombo admin2 layer to the map
Map.addLayer(boundary, {'color': 'green'}, 'Johor')

# Display the map
Map

## Visualize Landsat 9 RGB Imagery
Next, filter Landsat 9 surface reflectance imagery for the year 2024 over Johor State, creates a median composite, and displays a true-color RGB image on the interactive map.

In [ ]:
# Load Landsat 9 and filter by date, location, and cloud cover ≤ 30%
dataset = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterBounds(boundary) \
    .filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.lte('CLOUD_COVER', 30))

# Apply scale factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

dataset = dataset.map(apply_scale_factors)

# Compute the median composite
L9_median = dataset.median().clip(boundary)

# Visualization parameters for RGB
vis_rgb = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3
}

# Optional: Display NIR, SWIR, etc. for analysis
# vis_nir = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min': 0.0, 'max': 0.3}  # NIR-R-G

# Create and display the map
Map.addLayer(L9_median, vis_rgb, 'Landsat 9 RGB (2024)')
Map

## Visualize Sentinel-2 RGB Imagery
Next, filter Sentinel-2 surface reflectance imagery for the year 2024 over Johor State, creates a median composite, and displays a true-color RGB image on the interactive map.

In [ ]:
# Load and Filter Sentinel-2
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(boundary) \
    .filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .median() \
    .clip(boundary)

s2_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']  # RGB
}
Map.addLayer(s2, s2_vis, 'Sentinel-2 RGB (2023)')

# Display the map
Map

### # Generate XYZ Tile URL for QGIS Visualization
Next, retrieve the tile URL for displaying the Sentinel-2 image in QGIS as a web map layer.

In [ ]:
# Get tile URL for QGIS
map_id_dict = s2.getMapId(s2_vis)
tile_url = map_id_dict['tile_fetcher'].url_format
print("XYZ Tile URL for QGIS:")
print(tile_url)

XYZ Tile URL for QGIS:
https://earthengine.googleapis.com/v1/projects/ee-kamusoko-test/maps/e17d9f13757bcbaf581236f8878853a4-8497bc293cf272cb7a37e00e6e7e7ace/tiles/{z}/{x}/{y}


## Visualize Sentinel-1 VV, VH, and composite backscatter
Next, process Sentinel-1 radar imagery for 2024 over Johor, displaying the VV and VH polarization bands individually and as a pseudo-RGB composite.

In [ ]:
# Load and Filter Sentinel-1
s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(boundary) \
    .filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .median() \
    .clip(boundary)

# Display VV band only
Map.addLayer(s1.select('VV'), {'min': -20, 'max': 0}, 'Sentinel-1 VV')

# Display VH band only
Map.addLayer(s1.select('VH'), {'min': -25, 'max': -5}, 'Sentinel-1 VH')

# Display VV/VH Composite (VV-R, VH-G, VV-B)
composite_vis = {
    'min': -25,
    'max': 0,
    'bands': ['VV', 'VH', 'VV']  # Pseudo-RGB
}
Map.addLayer(s1, composite_vis, 'Sentinel-1 VV/VH Composite')

# Show map
Map

## Create ALOS PALSAR-2 ScanSAR
Next, disply ALOS PALSAR-2 (Advanced Land Observing Satellite – Phased Array type L-band Synthetic Aperture Radar). ALOS PALSAR-2  is a Japanese L-band radar sensor (Japan Aerospace Exploration Agency).

In [ ]:
# Load ALOS PALSAR-2 ScanSAR image collection and filter by boundary and date
collection = (
    ee.ImageCollection('JAXA/ALOS/PALSAR-2/Level2_2/ScanSAR')
    .filterBounds(boundary)
    .filterDate('2024-01-01', '2024-12-31')
)

# Compute median composites for HH and HV
Palsar_median_HH = collection.select('HH').median().clip(boundary).unitScale(0, 8000)

# Add PALSAR ScanScar layers to the map
Map.addLayer(Palsar_median_HH, {'min': 0, 'max': 1}, 'PALSAR Median HH')

# Show map
Map

## Display ESA land cover
Finally, display the ESA land cover map for 2021.

In [ ]:
# Load ESA WorldCover 2021 and clip to boundary
esa_lc = ee.Image("ESA/WorldCover/v200/2021").clip(boundary)

# Correct visualization with exact class values and colors
lc_vis = {
    'min': 10,
    'max': 100,
    'palette': [
        '006400',  # 10 - Tree cover
        'ffbb22',  # 20 - Shrubland
        'ffff4c',  # 30 - Grassland
        'f096ff',  # 40 - Cropland
        'fa0000',  # 50 - Built-up
        'b4b4b4',  # 60 - Bare / sparse vegetation
        'f0f0f0',  # 70 - Snow and ice
        '0064c8',  # 80 - Water
        '0096a0',  # 90 - Herbaceous wetland
        '00cf75',  # 95 - Mangroves
        'fae6a0'   # 100 - Moss and lichen
    ]
}

# Add to map
Map.addLayer(esa_lc, lc_vis, 'ESA WorldCover (2021)')

# Matching legend
legend_dict = {
    'Tree cover': '006400',
    'Shrubland': 'ffbb22',
    'Grassland': 'ffff4c',
    'Cropland': 'f096ff',
    'Built-up': 'fa0000',
    'Bare / sparse vegetation': 'b4b4b4',
    'Snow and ice': 'f0f0f0',
    'Permanent water bodies': '0064c8',
    'Herbaceous wetland': '0096a0',
    'Mangroves': '00cf75',
    'Moss and lichen': 'fae6a0'
}

Map.add_legend(title="ESA WorldCover 2021", legend_dict=legend_dict)
Map

## Lab Assignment
### Task
Using your study area of interest (country, province, or district), perform the following:
1. Display the Sentinel-2 data in QGIS.
2. Create land cover polygons in QGIS.